In [1]:
import yfinance as yf
import pandas as pd
from df_preprocessing import convert_to_pct
import os

import time

In [2]:
dva_spec = False

In [3]:
acl_18_ext_2016 = ['AAPL', 'ABB', 'ABBV', 'AEP', 'AMGN', 'AMZN', 'BA', 'BAC', 'BBL', 'BCH', 'BHP', 'BP', 'BRK-A', 'BSAC', 'BUD', 'C', 'CAT', 'CELG', 'CHL', 'CHTR', 'CMCSA', 'CODI', 'CSCO', 'CVX', 'D', 'DHR', 'DIS', 'DUK', 'EXC', 'FB', 'GD', 'GE', 'GOOG', 'HD', 'HON', 'HRG', 'HSBC', 'IEP', 'INTC', 'JNJ', 'JPM', 'KO', 'LMT', 'MA', 'MCD', 'MDT', 'MMM', 'MO', 'MRK', 'MSFT', 'NEE', 'NGG', 'NVS', 'ORCL', 'PCG', 'PCLN', 'PEP', 'PFE', 'PG', 'PICO', 'PM', 'PPL', 'PTR', 'RDS-B', 'REX', 'SLB', 'SNP', 'SNY', 'SO', 'SPLP', 'SRE', 'T', 'TM', 'TOT', 'TSM', 'UL', 'UN', 'UNH', 'UPS', 'UTX', 'V', 'VZ', 'WFC', 'WMT', 'XOM']

# stock list data

In [4]:
#daily interval
stock_ticker_dict = {}

for stock_name in acl_18_ext_2016:
    
    c_ticker = yf.Ticker(stock_name)
    c_data = c_ticker.history(start='2016-01-01', end='2024-01-01', interval='1d')
    stock_ticker_dict[stock_name] = [c_ticker, c_data]


    time.sleep(0.5)

ABB: No timezone found, symbol may be delisted
BBL: No timezone found, symbol may be delisted
CELG: No timezone found, symbol may be delisted
CHL: No timezone found, symbol may be delisted
FB: No timezone found, symbol may be delisted
HRG: No price data found, symbol may be delisted (1d 2016-01-01 -> 2024-01-01)
PCLN: No price data found, symbol may be delisted (1d 2016-01-01 -> 2024-01-01)
PICO: No timezone found, symbol may be delisted
PTR: No timezone found, symbol may be delisted
RDS-B: No timezone found, symbol may be delisted
SNP: No timezone found, symbol may be delisted
TOT: No timezone found, symbol may be delisted
UN: No timezone found, symbol may be delisted
UTX: No timezone found, symbol may be delisted


In [5]:
# stock_ticker_dict['ABB'][1].empty

#check empty

temp_l = []

for stock_name in acl_18_ext_2016:
    if not stock_ticker_dict[stock_name][1].empty:
        temp_l.append(stock_name)

acl_18_ext_2016 = temp_l


In [6]:
len(acl_18_ext_2016)

71

In [7]:
for stock_name in acl_18_ext_2016:

    pct_list = ['open', 'high', 'low', 'close']
    
    #original
    stock_ticker_dict[stock_name][1].drop(['Dividends', 'Stock Splits'], axis=1, inplace=True)
    stock_ticker_dict[stock_name][1] = stock_ticker_dict[stock_name][1].reset_index()
    stock_ticker_dict[stock_name][1] = stock_ticker_dict[stock_name][1].rename(columns=lambda x: x.lower())

    #datetime info change
    temp_dt = pd.to_datetime(stock_ticker_dict[stock_name][1].date)
    stock_ticker_dict[stock_name][1]['date'] = temp_dt.dt.strftime('%Y-%m-%d')
    #add the target
    if dva_spec:
        stock_ticker_dict[stock_name][1]['target'] = stock_ticker_dict[stock_name][1]['close'].copy()
        pct_list.append('target')
    

    #pct
    temp_data = stock_ticker_dict[stock_name][1].copy()
    convert_to_pct(temp_data, pct_list, drop_NA=True)
    temp_data = temp_data.reset_index().drop('index', axis=1)

    #temp_renaming for dva
    #temp_data.rename(columns={"open_pct_change": "open", "high_pct_change": "high", 'low_pct_change': "low", "close_pct_change": "close", "target_pct_change": "target"}, inplace=True)

    stock_ticker_dict[stock_name].append(temp_data)


    


    

In [8]:
# #test
# dtx = sp_500_data[sp_500_data['index_date'].isin(stock_ticker_dict['AAPL'][2]['date'])].reset_index().drop('index', axis=1)
# dtx_m = pd.concat([stock_ticker_dict['AAPL'][2], dtx], axis=1, join='outer')
# # dtx = dtx.rename(columns={'index_date':'date'})
# # dtx.date = pd.to_datetime(dtx.date)
# # stock_ticker_dict['AAPL'][2].date = pd.to_datetime(stock_ticker_dict['AAPL'][2].date)
# # dtx_m = pd.merge(stock_ticker_dict['AAPL'][2], index_match_df, on='date', how='left')

In [9]:
# display(dtx[-10:])
# display(dtx_m)
# acl_18_ext[0]

In [10]:

display(stock_ticker_dict[acl_18_ext_2016[0]][1])
display(stock_ticker_dict[acl_18_ext_2016[0]][2])

,date,open,high,low,close,volume
0,2016-01-04,23.324096,23.951468,23.185438,23.946920,270597600
1,2016-01-05,24.037844,24.060574,23.278635,23.346827,223164000
2,2016-01-06,22.858112,23.269541,22.701270,22.889935,273829600
3,2016-01-07,22.430770,22.760366,21.919327,21.923872,324377600
4,2016-01-08,22.401220,22.528512,21.994338,22.039799,283192000
...,...,...,...,...,...,...
2007,2023-12-22,194.931260,195.160978,192.724085,193.353287,37122800
2008,2023-12-26,193.363270,193.642911,192.584265,192.803986,28919300
2009,2023-12-27,192.244692,193.253399,190.846467,192.903839,48087700
2010,2023-12-28,193.892582,194.411923,192.923817,193.333298,34049900


,date,open,high,low,close,volume,open_pct_change,high_pct_change,low_pct_change,close_pct_change
0,2016-01-05,24.037844,24.060574,23.278635,23.346827,223164000,3.060129,0.455532,0.401963,-2.505933
1,2016-01-06,22.858112,23.269541,22.701270,22.889935,273829600,-4.907812,-3.287676,-2.480236,-1.956977
2,2016-01-07,22.430770,22.760366,21.919327,21.923872,324377600,-1.869540,-2.188158,-3.444492,-4.220469
3,2016-01-08,22.401220,22.528512,21.994338,22.039799,283192000,-0.131740,-1.018676,0.342215,0.528769
4,2016-01-11,22.496687,22.517144,22.126174,22.396671,198957600,0.426172,-0.050457,0.599410,1.619219
...,...,...,...,...,...,...,...,...,...,...
2006,2023-12-22,194.931260,195.160978,192.724085,193.353287,37122800,-0.469149,-0.847364,-0.273895,-0.554743
2007,2023-12-26,193.363270,193.642911,192.584265,192.803986,28919300,-0.804381,-0.777853,-0.072549,-0.284092
2008,2023-12-27,192.244692,193.253399,190.846467,192.903839,48087700,-0.578485,-0.201150,-0.902357,0.051790
2009,2023-12-28,193.892582,194.411923,192.923817,193.333298,34049900,0.857184,0.599484,1.088493,0.222628


In [11]:
save_raw = False
save_pct = True
save_pct_index = False


#export csv
folder_raw = 'EcmP_stock_L_2016_24_dva_spec/'
folder_pct = 'EcmP_stock_L_2016_24_spec_official_dva_spec/'
folder_pct_index = 'stock_us_pctindex/'

if not os.path.exists(folder_raw) and save_raw:
    os.makedirs(folder_raw)

if not os.path.exists(folder_pct) and save_pct:
    os.makedirs(folder_pct)

if not os.path.exists(folder_pct_index) and save_pct_index:
    os.makedirs(folder_pct_index)

for stock_name in acl_18_ext_2016:

    if save_raw:
        file_ori = folder_raw + stock_name+'.csv'
        stock_ticker_dict[stock_name][1].to_csv(file_ori, index=False)

    if save_pct:
        file_pct = folder_pct + stock_name+'_pct.csv'
        stock_ticker_dict[stock_name][2].to_csv(file_pct, index=False)

    if save_pct_index:
        file_pct_index = folder_pct_index + stock_name + '_pct_index.csv'
        stock_ticker_dict[stock_name][3].to_csv(file_pct_index, index=False)

1) apple 
2) Lockheed Martin 
3) Boeing 
4) AT&T
5) Netflix
6) tsmc
7) asml
8) intel
9) nvidia 
10) amd
11. HLX
12. XPOF
13. WGO
14. MLTX
15. WW
16. INSW
17. MLTX
18. msft
19. intc
20.  jpmc
21. amzn
22. tsla